In [1]:
import cv2
import numpy as np
import requests
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [2]:
haar_data = cv2.CascadeClassifier("data.xml")

In [10]:
capture = cv2.VideoCapture(0)
data = []
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 255), 4)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50, 50))
            if len(data)< 200:
                data.append(face)
        cv2.imshow("result", img)
        if cv2.waitKey(2) == 27 or len(data)>=200:
            break
capture.release()
cv2.destroyAllWindows()

In [8]:
np.save('without_mask.npy', data)

In [11]:
np.save("with_mask.npy", data)

In [12]:
with_mask = np.load("with_mask.npy")
without_mask = np.load("without_mask.npy")

In [13]:
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [14]:
X = np.r_[with_mask, without_mask]

In [15]:
labels = np.zeros(X.shape[0])
labels[200:] = 1.0
names = {0:'Mask', 1:'No Mask'}

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.25)

In [22]:
svm = SVC()
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)

In [25]:
accuracy_score(y_test, y_pred)

0.99